# Text Generation using OpenAI LLMs (GPT 3.5/GPT4)

- Installation & Importing Libraries
- Accessing OpenAI (Keys) & Setting up Client

---
- Chat Completion API (Basic Text Generation)
- Tokens
- Embeddings
- RAG
- Fine Tuning



- Time (75 minutes coding + 15 minutes QnA) 
- We'll also take questions at the end of each section
- Aim is to explore OpenAI features

## Installation & Importing Libraries

In [ ]:
%pip install --upgrade pip --quiet

In [ ]:
%pip install -r ../requirements.txt --quiet

In [ ]:
import openai #OpenAI python library
from openai import OpenAI #OpenAI Client
from configparser import ConfigParser #library to read the config file

import tiktoken #library to count tokens

import gradio as gr #library for gradio interface

from sklearn.metrics.pairwise import cosine_similarity #for calculating similarities between embeddings

from bs4 import BeautifulSoup #for extracting text from webpages
import requests

import PyPDF2 #for reading text from pdf 

from langchain.document_loaders import TextLoader #to load text for RAG
from langchain.text_splitter import RecursiveCharacterTextSplitter #to chunk data for RAG
from langchain.embeddings.openai import OpenAIEmbeddings #to crate embeddings for RAG
from langchain.vectorstores import FAISS # to store embeddings in a vector index
from sklearn.model_selection import train_test_split #for model finetuning

import json #for creating finetuning files

import numpy as np
from sklearn.cluster import KMeans

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## 
___

## Accessing OpenAI (Keys) & Setting up Client

__Question__ : Have you procured your OpenAI API key?

If not,  create one here - https://platform.openai.com/api-keys

Also, check if you have sufficient balance - https://platform.openai.com/account/billing/overview

__Step 1__ : Read and set the OpenAI API key in the environment

In [ ]:
config_object = ConfigParser()
config_object.read("../config.ini")
openai.api_key = config_object["OPENAI"]["openai_key"] #read the api key from the config file

In [ ]:
#openai.api_key="<Your API Key>"

There are several ways of storing the API key in the environment. You may choose as per your preference or your organisation's policy

__Step 2__: Initialize the OpenAI client. This serves as an interface to interact with OpenAI's services and APIs.

In [ ]:
client = OpenAI(api_key=openai.api_key)

And we're ready! Let's try and make the first call!

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)
print(response.choices[0].message.content)

## 
---

## Chat Completion API (Basic Text Generation)

### Introduction

<span style="font-size: 20px; color: orange">>><b>Generative AI, and LLMs specifically, is a General Purpose Technology that is useful for a variety of applications</b></span>

<span style="font-size: 16px;"><i>"LLMs can be, generally, thought of as a next word prediction model"</i></span>

<span style="font-size: 16px; color: blue"><b>What is an LLM?</b></span>

- LLMs are __machine learning models__ that have learned from __massive datasets__ of human-generated content, finding statistical patterns to replicate human-like abilities.

- __Foundation models__, also known as base models, have been trained on trillions of words for weeks or months using extensive compute power. These models have __billions of parameters__, which represent their memory and enable sophisticated tasks.

- __Interacting with LLMs differs from traditional programming paradigms. Instead of formalized code syntax, you provide natural language prompts to the models__.

- When you pass a __prompt__ to the model, it predicts the next words and generates a __completion__. This process is known as __inference__.


<span style="font-size: 16px; color: blue"> <b>Prompts, Completions and Inference!</b></span>

<img src="../Assets/Images/LLM Inference.png" width=800>

<span style="font-size: 16px; color: blue"> <b>Available OpenAI models</b></span>

---

__GPT 4__

__<u>(Production)</u>__

<u>Name        | Context Window    | Cut-off date      | Snapshot</u>

__gpt-4__       | 8,192 tokens      | Up to Sep 2021    | gpt-4-0613

__gpt-4-32k__   | 32,768 tokens     | Up to Sep 2021    | gpt-4-32k-0613

<b><u>(Preview)</b></u>

__gpt-4-turbo-preview__     | 128,000 tokens | Up to Dec 2023    | gpt-4-1106-preview

__gpt-4-vision-preview__    | 128,000 tokens | Up to Apr 2023    | gpt-4-1106-vision-preview

---

__GPT 3.5__

__gpt-3.5-turbo__ | 16,385 tokens | Up to Sep 2021 | gpt-3.5-turbo-1106

__gpt-3.5-turbo-instruct__ | 4,096 tokens | Up to Sep 2021

---

For more details, visit the official documentation -> https://platform.openai.com/docs/models

---

<span style="font-size: 14px; color: orange">__IMP__ : __"model"__ is passed as a parameter in the chat completions API</span>

---





<span style="font-size: 16px; color: blue"> <b>OpenAI Chat Messages (Prompt) Structure</b></span>

<span style="font-size: 16px;"><u>Role</u></span> : OpenAI allows for *three* roles/personas - 
1. __System__ : The overarching constraints/definitions/intructions of the system that the LLM should "remember"
2. __User__ : Any instruction a user wants to pass to the LLM
3. __Assistant__ : The response from the LLM

<span style="font-size: 16px;"><u>Content</u></span> : Any message or "prompt" of these personas are passes as "Content"



__Why is this important?__ : Makes it easier to adapt an LLM to store conversation history.

---

<span style="font-size: 14px; color: orange">__IMP__ : __"role"__ and __"content"__ are passed as a dictionary in the __"messages"__ parameter in the chat completions API </span>

---

__Let's try__!

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant knowledgeable in the field of Cricket."},
    {"role": "user", "content": "When did Australia win their first Cricket World Cup?"}
  ]
)

In [ ]:
print(response.choices[0].message.content)

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant knowledgeable in the field of Cricket."},
    {"role": "user", "content": "When did Australia win their first Cricket World Cup?"},
    {"role": "assistant", "content": "Australia won their first Cricket World Cup in the year 1987. They defeated England in the final to clinch their maiden title in the tournament."},
    {"role": "user", "content": "How much did they score?"}
  ]
)

from pprint import pprint

pprint(response.choices[0].message.content)

### Chat API Parameters

<span style="font-size: 20px; color: orange">>><b>"model"</b> and <b>"messages"</b> are the two required API parameters</span>

<span style="font-size: 16px; color: blue"> There are several other optional parameters that help configure the response</span>

---

__n__ : Number of responses you want the LLM to generate for the instriction

__max_tokens__ : Maximum number of tokens you want to restrict the Inference to (This includes both the prompt/messages and the completion)

__temperature__ : Temperature controls the "randomness" of the responses. Higher value increases the randomness; lower value makes the output deterministic (value between 0 and 2)

__top_p__ : The model considers the results of the tokens with top_p probability mass (value between 0 and 1)


<img src="../Assets/Images/Temperature - Top P.png" width=600>

<span style="font-size: 14px; color: orange"> __IMP__ : It is recommended to configure either one of "temperature" and "top_p" but not both</span>


__frequency_penalty__ : Penalize new tokens based on their existing frequency in the text so far *(Value between -2 and 2)*

__presence_penalty__ : Penalize new tokens based on whether they appear in the text so far *(Value between -2 and 2)*

__logprobs__ : Flag to return log probability of the generated tokens *(True/False)*

__logit_bias__ : Parameter to control the presence of particular tokens in the output *(Value between -100 and 100)

__response_format__ : Response of the model can be requested in a particular format *(Currently : JSON and Text)*

__seed__ : Beta feature for reproducible outputs (setting a seed value may produce the same output repeatedly)

__stop__ : End of Sequence tokens that will stop the generation

__stream__ : To receive partial message deltas *(True/False)*

__user__ : ID representing end user (This helps OpenAI detect abuse. May be mandatory for higher rate limits)

__tools__ : used in function calling

__tool_choice__ : used in function calling

---

In [ ]:
def gpt_call(model:str="gpt-3.5-turbo",prompt:str="Have I provided any input",n:int=1,max_tokens:int=100,temperature:float=0.5,presence_penalty:float=0):


    response = client.chat.completions.create(
    model=model,
    messages=[
       {"role": "user", "content": prompt}
    ],
    max_tokens=max_tokens,
    temperature=temperature,
    presence_penalty=presence_penalty,
    n=n
    )
    
    output=''

    for i in response.choices:
        output+=str(i.message.content)+'\n------------\n'
        
    

    return output


    


In [ ]:
print(gpt_call(prompt="Write a title for a workshop on openai API",n=2,temperature=1))

In [ ]:

def text_to_uppercase(model,n,max_tokens,temperature,presence_penalty,prompt):
    return str(type(n)) + str(type(temperature))

model=gr.Radio(["gpt-4","gpt-3.5-turbo"], label="Select Model")
n=gr.Radio([1,2,3], label="Number of Responses")
max_tokens=gr.Slider(minimum=10, maximum=500, label="Maximum Tokens")
temperature=gr.Slider(minimum=0.0, maximum=1.0, label="Temperature")
prompt=gr.Text(label="Prompt")


iface = gr.Interface(fn=gpt_call, inputs=[model,prompt,n,max_tokens,temperature], outputs="text")
iface.launch()


In [ ]:
iface.close()

### Decoding the Response Object

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant knowledgeable in the field of Cricket."},
    {"role": "user", "content": "When did Australia win their first Cricket World Cup?"},
    {"role": "assistant", "content": "Australia won their first Cricket World Cup in the year 1987. They defeated England in the final to clinch their maiden title in the tournament."},
    {"role": "user", "content": "How much did they score?"}
  ]
)

In [ ]:
print((response.model_dump_json(indent=5)))


<img src="../Assets/Images/Chat Completion Object.png" width=1000>


### Streaming

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant knowledgeable in the field of Cricket."},
    {"role": "user", "content": "When did Australia win their first Cricket World Cup?"},
    {"role": "assistant", "content": "Australia won their first Cricket World Cup in the year 1987. They defeated England in the final to clinch their maiden title in the tournament."},
    {"role": "user", "content": "How much did they score?"}
  ],
  stream=True
)

for chunk in response:
  print(chunk.choices[0].delta.content)


In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "Write an email requesting for a leave of absence"}
  ],
  stream=True
)

for chunk in response:
  print(chunk.choices[0].delta.content)

### JSON

In [ ]:
prompt="generate the entire text for a blog on a cricket match. \"Title\" is a catchy and attractive title of the blog. The \"Heading\" is the heading for each point in the blog and the \"Body\" is the text for that heading.Output in a json structure"

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "user", "content": prompt}
  ],
  response_format={ "type": "json_object" }
)

print(response.choices[0].message.content)

## 

---

## Tokens

<span style="font-size: 20px; color: orange">>><b>Tokens are the fundamental units of NLP</b></span>

<span style="font-size: 16px; color: blue"><b>These units are typically words, punctuation marks, or other meaningful substrings that make up the text</b></span>

Counting the number of tokens becomes important because - 
- Number of Tokens determine the amount of computation required and hence the cost you incur
- Context Window or the maximum number of tokens an LLM can process in one go is limited


### Counting Tokens

In [ ]:
####num_tokens_from_string function to count number of tokens in a text string
####uses tiktoken to count number of tokens in a text string
####parameters: "string" is the text string, "encoding_name" is the encoding name to be used by tiktoken
####returns: num_tokens->number of tokens in the text string
####This function is used within extract_data, extract_page, extract_YT, extract_audio, extract_image functions
def num_tokens_from_string(string: str, encoding_name="cl100k_base") -> int: #### Function to count number of tokens in a text string ####
    encoding = tiktoken.get_encoding(encoding_name) #### Initialize encoding ####
    return len(encoding.encode(string)) #### Return number of tokens in the text string ####

In [ ]:
num_tokens_from_string("Hello how are you?")

In [ ]:
with open("../Assets/Data/alice_in_wonderland.txt") as f:
    AliceInWonderland = f.read()

num_tokens_from_string(AliceInWonderland)

### Pricing



__gpt-3.5-turbo-0125__	    |  PROMPT - $0.50 / 1M tokens   |   RESPONSE - $1.50 / 1M tokens

__gpt-4__	                |   PROMPT - $30.00 / 1M tokens	|   RESPONSE - $60.00 / 1M tokens

__gpt-4-turbo__             |	PROMPT - $10.00 / 1M tokens	|   RESPONSE - $30.00 / 1M tokens

## 

---

## Embeddings

### Introduction

<span style="font-size: 20px; color: orange">>><b>Embeddings are vector representations of data that capture meaningful relationships between entities</b></span>

<span style="font-size: 16px; color: blue"><b>These units are typically words, punctuation marks, or other meaningful substrings that make up the text</b></span>

- All Machine Learning/AI models work with numerical data. Before the performance of any operation all text/image/audio/video data has to be transformed into a numerical representation

- As a general definition, embeddings are data that has been transformed into n-dimensional matrices for use in deep learning computations.

<img src="../Assets/Images/Embeddings.png" width=800>

<span style="font-size: 16px; color: blue"><b>OpenAI Embeddings</b></span>

__text-embedding-3-small__	| $0.02 / 1M tokens

__text-embedding-3-large__	| $0.13 / 1M tokens

__ada v2__	| $0.10 / 1M tokens

In [ ]:

embeddings=client.embeddings.create(
  model="text-embedding-3-small",
  input="The food was delicious",
  encoding_format="float",
)


In [ ]:
print(embeddings.model_dump_json(indent=4))

In [ ]:
embeddings=client.embeddings.create(
  model="text-embedding-3-small",
  input=["The food was delicious","The ambience was nice","The service was ordinary"],
  encoding_format="float",
  dimensions=10
)

In [ ]:
print(embeddings.model_dump_json(indent=4))

### Text Search

In [ ]:
embeddings_q=client.embeddings.create(
  model="text-embedding-3-small",
  input="food",
  encoding_format="float",
  dimensions=10
)

In [ ]:
query=embeddings_q.data[0].embedding

In [ ]:
query

In [ ]:
d1=embeddings.data[0].embedding #"The food was delicious"
d2=embeddings.data[1].embedding #"The ambience was nice"
d3=embeddings.data[2].embedding #"The service was ordinary"

<img src="https://storage.googleapis.com/lds-media/images/cosine-similarity-vectors.original.jpg">

In [ ]:
cosine_similarity([query],[d1])

In [ ]:
cosine_similarity([query],[d2])

In [ ]:
cosine_similarity([query],[d3])

__1300+ Towards DataScience Medium Articles Dataset__

Data Source - https://www.kaggle.com/datasets/meruvulikith/1300-towards-datascience-medium-articles-dataset

In [ ]:
data=pd.read_csv("../Assets/Data/medium.csv")

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
trunc_data=data.iloc[0:100,:]

In [ ]:
trunc_data.shape

In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [ ]:
trunc_data['embedding'] = trunc_data.Title.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))

In [ ]:
trunc_data

In [ ]:
search_string="Deep Learning"

In [ ]:
search_embedding=get_embedding(search_string)

In [ ]:
trunc_data['relevance'] = trunc_data.embedding.apply(lambda x: float(cosine_similarity([search_embedding],[x])))

In [ ]:
trunc_data

In [ ]:
trunc_data.sort_values(by="relevance",ascending=False).iloc[0:10,:]

### Clustering

In [ ]:

matrix = np.vstack(trunc_data.embedding.values)
n_clusters = 4

kmeans = KMeans(n_clusters = n_clusters, init='k-means++', random_state=42)
kmeans.fit(matrix)
trunc_data['Cluster'] = kmeans.labels_

In [ ]:
trunc_data

In [ ]:
trunc_data["Prompt"] = trunc_data["Title"] + " belongs to Cluster number " + trunc_data["Cluster"].astype(str)

In [ ]:
info=''
for i in trunc_data["Prompt"]:
    info+=i+'\n'


In [ ]:
print(info)

In [ ]:
prompt="""Below is information of blog titles grouped into clusters. There are four clusters. Come up with Names for these four clusters basis the titles present in them\n\n"""+info

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": prompt}
  ]
)

In [ ]:
print(response.choices[0].message.content)

## 
---

## RAG

### Introduction

<span style="font-size: 20px; color: orange">>><b>Users look at LLMs for knowledge and wisdom, yet LLMs are sophisticated predictors of what word comes next</b></span>

<span style="font-size: 16px; color: blue"><b>Hallucinations and Restricted "Parameteric" Memory are the biggest drawbacks of LLMs</b></span>

- Hallucinations - Very confidently, provide incorrect information.
- Missing Knowledge - Not having information (information available only that is available the training data)






In [ ]:
prompt="What is amica developed by Portable?"

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": prompt}
  ]
)

In [ ]:
pprint(response.choices[0].message.content)

<span style="font-size: 14px; color: blue"><b>Retrieval Augmented Generation or RAG seems to solve these problems</b></span>


<img src="../Assets/Images/RAG.png" width=800>

### Retrieval

In [ ]:


def extract_page(link): #### Function to extract text from weblink ####
    address=link #### Store weblink in address variable ####
    response=requests.get(address) #### Get response from weblink using requests ####
    soup = BeautifulSoup(response.content, 'html.parser') #### Parse response using BeautifulSoup ####
    text=soup.get_text() #### Extract text from parsed response ####
    lines = filter(lambda x: x.strip(), text.splitlines()) #### Filter out empty lines ####
    website_text = "\n".join(lines) #### Join lines to form text ####
    return website_text

In [ ]:
text=extract_page("https://portable.com.au/work/amica")

### Augmentation

In [ ]:
augmented_prompt=f"You have been provided a context about below. Based only on the context answer the following question \n\n context : {text}\n\n question : {prompt}"

### Generation

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": augmented_prompt}
  ]
)

In [ ]:
pprint(response.choices[0].message.content)

---

In [ ]:
encoding_name="cl100k_base"
encoding = tiktoken.get_encoding(encoding_name)
len(encoding.encode(text))

In [ ]:

reader = PyPDF2.PdfReader("../Assets/Data/InnovatorsDilemma.pdf")
pages = reader.pages
# get all pages data
text = "".join([page.extract_text() for page in pages])

In [ ]:
len(encoding.encode(text))

### RAG At Scale

In [ ]:
def split_text_and_create_embeddings(text): #### Function to create embeddings from text ####
    with open('../Assets/Data/temp.txt','w') as f: #### Write text to a temporary file ####
         f.write(text) #### Write text to a temporary file ####
         f.close() #### Close temporary file ####
    loader=TextLoader('../Assets/Data/temp.txt') #### Load temporary file using TextLoader ####
    document=loader.load() #### Extract text from temporary file ####
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=2000) #### Initialize text splitter to split text into chunks of 10000 tokens ####
    docs = text_splitter.split_documents(document) #### Split document into chunks of 10000 tokens ####
    num_emb=len(docs) #### Count number of embeddings ####
    embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key) #### Initialize embeddings ####
    db = FAISS.from_documents(docs, embeddings) #### Create embeddings from text ####
    return db, num_emb #### Return database with embeddings and number of embeddings ####

In [ ]:
vector_db, num_emb=split_text_and_create_embeddings(text)

In [ ]:
print(num_emb)

In [ ]:
vector_db.save_local(folder_path="../Assets/Data")

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)

In [ ]:
local_vectors=FAISS.load_local(folder_path="../Assets/Data/",embeddings=embeddings,allow_dangerous_deserialization=True)

In [ ]:
def search_context(db,query): ###### search_context function
     defin=db.similarity_search(query) ###### call the FAISS similarity_search function that searches the database for the most relevant section to the user question and orders the results in descending order of relevance
     return defin[0].page_content ###### return the most relevant section to the user question

In [ ]:
prompt="Why do great firms fail?"

In [ ]:
context=search_context(local_vectors,prompt)

In [ ]:
augmented_prompt=f"You have been provided a context in [[[]]] and a prompt below. Respond to the prompt only from the context. If the information is not present in the context, respond with \"I don't know\"\nContext : [[[{context}]]]\nPrompt : {prompt}"

In [ ]:
pprint(client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": augmented_prompt}
  ]
).choices[0].message.content)

## 
---

## Fine Tuning

### Introduction



<img src="../Assets/Images/SFT.png" width=1000>

Taking a general purpose model and train it to perform a specialized/specific task

- Hallucinations
- RAG Misses
- Learn New Information (When data size is large)
- Cost Optimization
- Privacy

**Challenges**

- Need quality data
- Upfront cost
- Expertise

There are three broad steps in LLM finetuning -

1. Data Preparation (for the specific task/use case)
2. Training (chosen training methodology)
3. Evaluation

### Data Preparation

In [ ]:
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}

In [ ]:
data_folder_path='../Assets/Data/'
training_data_file='fine_tuning_data.csv'

In [ ]:
data_for_finetuning=pd.read_csv(data_folder_path+training_data_file)

In [ ]:
data_for_finetuning.head()

In [ ]:
data_for_finetuning.shape

In [ ]:
train_data, test_data = train_test_split(data_for_finetuning, test_size=0.2, random_state=42)

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
prompt_end="<--"
completion_end="-->"

In [ ]:
def convert_data_for_turbo(data,file):
  with open(file,'w') as f:
    for _,rows in data.iterrows():
      prompt=rows['prompt']+prompt_end
      completion=rows['completion']+completion_end
      json_line = {'messages': [{'role': 'system',
                                'content': ''},
                                {'role': 'user',
                                 'content': prompt},
                                {'role': 'assistant',
                                'content': completion}]}
      f.write(json.dumps(json_line) + '\n')

  print(f'JSONlines file "{file}" has been created.')

In [ ]:
train_file_turbo_name='train_turbo.jsonl'
test_file_turbo_name='test_turbo.jsonl'

In [ ]:
convert_data_for_turbo(train_data,data_folder_path+train_file_turbo_name)
convert_data_for_turbo(test_data,data_folder_path+test_file_turbo_name)

In [ ]:
client.files.create(
                file=open(data_folder_path+train_file_turbo_name, "rb"),
                purpose='fine-tune'
)

In [ ]:
client.files.create(
                file=open(data_folder_path+test_file_turbo_name, "rb"),
                purpose='fine-tune'
)

### Training

In [ ]:
client.fine_tuning.jobs.create(
              training_file="file-6AUxqwaqG16AFy1PVLEGFKZI",
              validation_file="file-7GByyQB3nnXSKZtgLQt542UU",
              model="gpt-3.5-turbo-0125",
              suffix="Workshop",
              hyperparameters={"n_epochs":1,
                               "batch_size":1,
                               }
              )

In [ ]:
print(client.fine_tuning.jobs.list(limit=2).model_dump_json(indent=5))

In [ ]:
print(client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-OP2LgxuFTe4UxHoxYbYzG7Vw", limit=2).model_dump_json(indent=5))

### Evaluation

In [ ]:
test_data.iloc[10]["prompt"]

In [ ]:
prompt=str(test_data.iloc[10]["prompt"])+prompt_end

In [ ]:
response = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:artificial-kimtelligence:workshop:94dD2dNr",
  messages=[
            {'role': 'system','content': ''},
            {"role": "user", "content": prompt}
  ]
)

In [ ]:
print(response.choices[0].message.content[:-3])

In [ ]:
response = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:artificial-kimtelligence:workshop:94dD2dNr",
  messages=[
    {'role': 'system','content': ''},
    {"role": "user", "content": "explain How Homomorphic Encryption Works"}
  ]
)

print(response.choices[0].message.content)

##

---

<img src="../Assets/Images/profile.png" width=50> [Hi! I'm Abhinav!](https://www.linkedin.com/in/abhinav-kimothi/)

<span style="font-size: 20px; color: orange">>><b>Hope to stay connected!</b></span>


 
[![GitHub followers](https://img.shields.io/github/followers/abhinav-kimothi?label=Follow&style=social)](https://github.com/abhinav-kimothi)
[![Me](https://img.shields.io/badge/Medium-8A2BE2)](https://medium.com/@abhinavkimothi)
[![LIn](https://img.shields.io/badge/LinkedIn-blue)](https://www.linkedin.com/in/abhinav-kimothi/)
[![Mail](https://img.shields.io/badge/eMail-green)](mailto:abhinav.kimothi.ds@gmail.com)
[![Twitter Follow](https://img.shields.io/twitter/follow/@?style=social)](https://twitter.com/abhinav_kimothi)


<span style="font-size: 20px; color: orange">>><b>Also, read these for more details on Generative AI!</b></span>



<a href="https://abhinavkimothi.gumroad.com/l/GenAILLM">
    <img src="https://public-files.gumroad.com/jsdnnne2gnhu61f6hrdprwx2255i" width=150>
</a><a href="abhinavkimothi.gumroad.com/l/RAG">
    <img src="https://public-files.gumroad.com/v17k9tp2fnbbtg8iwoxt4m3xgivq" width=150>
</a><a href="abhinavkimothi.gumroad.com/l/GenAITaxonomy">
    <img src="https://public-files.gumroad.com/a730ysxb7a928bb5xkz6fuqabaqp" width=150>
</a>



---

<img src="../Assets/Images/KCBAI.png" height =300>